In [7]:
import os, sys
import django
from datetime import datetime
sys.path.append('../') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from edc_appointment.models import Appointment as CaregiverAppointment

In [8]:
# Query participants who have been sequentially enrolled
sq_enrolled = Cohort.objects.values(
    'subject_identifier').annotate(cohort_count=Count('subject_identifier')).filter(cohort_count__gt=1)
child_identifiers = sq_enrolled.values_list('subject_identifier', flat=True)
child_identifiers = set(child_identifiers)

In [9]:
fixed_appts = []
for identifier in child_identifiers:
    cohorts = Cohort.objects.filter(
        subject_identifier=identifier).order_by('assign_datetime')
    child_consent = CaregiverChildConsent.objects.filter(
        subject_identifier=identifier).latest('consent_datetime')
    schedules = CohortSchedules.objects.filter(cohort_name=cohorts[1].name,
                                               child_count=child_consent.caregiver_visit_count)
    caregiver_identifier = child_consent.subject_consent.subject_identifier
    for schedule in schedules:
        model_cls = django_apps.get_model(schedule.onschedule_model)
        try:
            model_obj = model_cls.objects.get(
                child_subject_identifier=identifier,
                schedule_name=schedule.schedule_name,)
        except model_cls.DoesNotExist:
            continue
        else:
            appts = CaregiverAppointment.objects.filter(
                subject_identifier=caregiver_identifier,
                schedule_name=model_obj.schedule_name,
                appt_status='new')
            for appt in appts:
                try:
                    prev_schedule = CohortSchedules.objects.get(cohort_name=cohorts[0].name,
                                                                child_count=schedule.child_count,
                                                                schedule_type=schedule.schedule_type)
                except CohortSchedules.DoesNotExist:
                    continue
                else:
                    old_appts = CaregiverAppointment.history.filter(
                        subject_identifier=caregiver_identifier,
                        schedule_name=prev_schedule.schedule_name,
                        visit_code__startswith=appt.visit_code[:4],
                        visit_code_sequence=appt.visit_code_sequence)
                    if old_appts.exists():
                        old_appt = old_appts.latest('history_date')
                        old_appt_dt = old_appt.appt_datetime.date()
                        appt_dt = appt.appt_datetime.date()
                        if appt_dt < old_appt_dt:
                            fixed_appts.append({'subject_identifier': caregiver_identifier,
                                                'prev_schedule': prev_schedule.schedule_name,
                                                'current_schedule': schedule.schedule_name,
                                                'old_appt': old_appt.visit_code,
                                                'new_appt': appt.visit_code,
                                                'current_date': appt_dt,
                                                'reschedule_date': old_appt_dt})
                            appt.appt_datetime = old_appt.appt_datetime
                            appt.appt_type = old_appt.appt_type
                            appt.appt_reason = old_appt.appt_reason
                            appt.comment = old_appt.comment
                            appt.save()

In [10]:
fixed_appts

[{'subject_identifier': 'B142-040990432-2',
  'prev_schedule': 'a_fu_quarterly1_schedule1',
  'current_schedule': 'b_sec_fu_quart1_schedule1',
  'old_appt': '3002M',
  'new_appt': '3002S',
  'current_date': datetime.date(2023, 7, 31),
  'reschedule_date': datetime.date(2023, 9, 28)},
 {'subject_identifier': 'B142-040990476-9',
  'prev_schedule': 'a_fu_quarterly1_schedule1',
  'current_schedule': 'b_sec_fu_quart1_schedule1',
  'old_appt': '3003M',
  'new_appt': '3003S',
  'current_date': datetime.date(2023, 10, 3),
  'reschedule_date': datetime.date(2023, 11, 24)},
 {'subject_identifier': 'C142-040990956-0',
  'prev_schedule': 'a_fu_quarterly1_schedule1',
  'current_schedule': 'b_sec_fu_quart1_schedule1',
  'old_appt': '3003M',
  'new_appt': '3003S',
  'current_date': datetime.date(2023, 10, 17),
  'reschedule_date': datetime.date(2023, 12, 1)},
 {'subject_identifier': 'CR142-040990257-3',
  'prev_schedule': 'b_quarterly1_schedule1',
  'current_schedule': 'c_quarterly1_schedule1',
  'ol